In [32]:
import numpy
import os
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [28]:
cd ./American_dad_script/

[WinError 2] Le fichier spécifié est introuvable: './American_dad_script/'
C:\Users\berte\Desktop\M2_AIC\NLP\Projet\American_dad_script


In [92]:
full_text = []

for filename in os.listdir(os.getcwd()):
    raw_text = open(filename, 'r', encoding='utf-8').read()
    full_text.append(raw_text.lower())


In [93]:
flat_list = [item for sublist in full_text for item in sublist]

In [94]:
flat_list = flat_list[:150000]

In [95]:
# create mapping of unique chars to integers
chars = sorted(list(set(flat_list)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [96]:
for key, value in char_to_int.items() :
    print (key, value)


  0
! 1
" 2
# 3
$ 4
% 5
& 6
' 7
, 8
- 9
. 10
/ 11
0 12
1 13
2 14
3 15
4 16
5 17
6 18
7 19
8 20
9 21
: 22
? 23
[ 24
] 25
a 26
b 27
c 28
d 29
e 30
f 31
g 32
h 33
i 34
j 35
k 36
l 37
m 38
n 39
o 40
p 41
q 42
r 43
s 44
t 45
u 46
v 47
w 48
x 49
y 50
z 51
à 52
á 53
ä 54
é 55
ó 56


In [97]:
n_chars = len(flat_list)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  150000
Total Vocab:  57


In [98]:
...
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = flat_list[i:i + seq_length]
	seq_out = flat_list[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  149900


In [99]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [100]:
...
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [101]:
...
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [103]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
149900/149900 [==============================] - 1078s 7ms/step - loss: 2.9576

Epoch 00001: loss improved from inf to 2.95756, saving model to weights-improvement-01-2.9576.hdf5
Epoch 2/20
   256/149900 [..............................] - ETA: 22:32 - loss: 2.9216

KeyboardInterrupt: 

In [62]:
import sys

In [104]:
...
# load the network weights
filename = "weights-improvement-01-2.9576.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [105]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [107]:
...
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
" ay the 14th? - yes. yes, it is. good god! how could i forget a date so dear to my heart? it's the bi "
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

KeyboardInterrupt: 